# Perhitungan SPI-1 dan SPI-3 dari Data CHIRPS
Notebook ini menghitung SPI-1 dan SPI-3 berdasarkan metode z-score dari CHIRPS bulanan.
- SPI-1: Z-score dari CHIRPS bulanan
- SPI-3: Z-score dari rata-rata CHIRPS 3 bulan berturut-turut


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Ganti path sesuai lokasi file
file_path = 'Lembang_Merged_Timeseries.csv'
df = pd.read_csv(file_path)
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.to_period('M').dt.to_timestamp()

df.sort_values(by=['lat', 'lon', 'month'], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

In [ ]:
# Fungsi untuk menghitung SPI per lokasi

def calculate_spi(group):
    group = group.sort_values('month').copy()
    
    mu = group['CHIRPS'].mean()
    sigma = group['CHIRPS'].std()
    group['SPI_1'] = (group['CHIRPS'] - mu) / sigma

    group['CHIRPS_3mo'] = group['CHIRPS'].rolling(window=3, min_periods=3).mean()
    group['SPI_3'] = (group['CHIRPS_3mo'] - mu) / sigma

    return group

In [ ]:
# Hitung SPI per titik koordinat
spi_result = df.groupby(['lat', 'lon'], group_keys=False).apply(calculate_spi)
spi_result.drop(columns='CHIRPS_3mo', inplace=True)
spi_result.head()

In [ ]:
# Simpan hasil ke CSV
output_path = 'Lembang_Merged_SPI_Reconstructed.csv'
spi_result.to_csv(output_path, index=False)
print(f'Hasil disimpan ke {output_path}')